<a href="https://www.kaggle.com/code/jy2040/random-forest?scriptVersionId=94539086" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# importing required libraries
from sklearn.tree import DecisionTreeClassifier
from random import randrange
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.utils import shuffle
import random

1. complete the following function that creates subsample of a dataset with replacement

In [2]:
# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio): 
    # WRITE YOUR CODE HERE
    # IF NUMBER OF ROWS IN CURRENT DATASET IS N, THE OUTPUT DATASET SHOULD HAVE N*RATIO NUMBER OF ROWS SELECTED RANDOMLY WITH REPLACEMENT FROM THE ORIGINAL DATASET
    # INPUT AND OUTPUT DATATYPE SHOULD BE PANDAS DATAFRAME
    return dataset.sample(frac=ratio, replace=True, random_state=1) # SAMPLED DATASET WITH REPLACEMENT WITH THE SIZE OF GIVEN RATIO

2. complete the following function that creates subsample of the dataset with feature size reduced as per the given ratio

In [3]:
# Create a random selection of n features
def subsample2(dataset, ratio):
    # WRITE YOUR CODE HERE
    # THE OUTPUT DATASET SHOULD HAVE SAME NUMBER OF ROWS AS THE INPUT DATASET
    # IF NUMBER OF FEATURES IN THE INPUT DATASET IS N, THEN THE NUMBER OF FEATURES IN THE OUTPUT DATASET SHOULD BE N*ratio
    # THE INPUT AND OUTPUT DATATYPE SHOULD BE PANDAS DATAFRAME
    
    features=dataset.columns.tolist()
    features.remove('target')
    ret = list()

    while len(ret) <= len(features)*ratio:
        
#         selecting random features from the dataset
        idx=random.randrange(0,len(features))
        
#         adding it to the list
        ret.append(features[idx])
        
#         deleting selected features to avoid duplicates
        del features[idx]
        
    ret.append('target')
    
    return ret,dataset[ret] # RETURN 2 ITEMS, 1ST IS THE SELECTED COLUMNS (DATATYPE LIST) AND 2ND IS THE REDUCED DATAFRAME

3. Perform train test split without using sklearn (complete the following function)

In [4]:
# Splitting dataset into train and test
import copy
def split_train_test(dataset, test_size):
    # WRITE YOUR CODE HERE
    # DATASET IS THE INPUT DATAFRAME
    # TEST_SIZE IS THE SIZE OF TESTING SET IN FORM OF FRACTION BETWEEN 0 AND 1
    # OUTPUT TWO DATAFRAMES, TRAIN AND TEST DATA
    
    temp_df = dataset.copy()
    
#     suffling the dataset instances
    temp_df=shuffle(temp_df).reset_index(drop=True)
    
    train_size = int(len(temp_df)*(1-test_size))
    test_size = int(len(temp_df) * (1 - test_size))
    train_data = temp_df.head(train_size).reset_index(drop=True)
    test_data=temp_df.head(test_size).reset_index(drop=True)
    
    return train_data, test_data

4. Perform training using random forest algorithm by completing the function below

In [5]:
# Random Forest Algorithm
def random_forest_train(train, n_trees, max_depth, sample_size, n_features_ratio):
    # WRITE YOUR CODE HERE
    # TRAIN IS THE TRAINING DATASET THAT YOU GET FROM THE OUTPUT OF YOUR TRAIN TEST SPLIT
    # N_TREES IS NUMBER OF TREES IN THE FOREST
    # MAX_DEPTH IS THE MAX_DEPTH OF EACH TREE IN THE FOREST
    # SAMPLE SIZE IS THE RATIO OF ROWS TO BE SENT TO EACH TREE (VALUE BETWEEN 0 AND 1)
    # N_FEATURES_RATIO IS THE RATIO OF FEATURES TO BE SENT TO EACH TREE
    # USE SKLEARN TREES TO SIMLIFY THE EXERCISE
    
    tModel_features= list()
    for tree in range(0,n_trees):
        
        dTree = DecisionTreeClassifier(random_state=0)
        
        
        subsample_df = subsample(train,sample_size)
        features, subsample2_df = subsample2(subsample_df,n_features_ratio)
        X=subsample2_df.drop(columns=['target'])
        y=subsample2_df['target']
        dTree.fit(X,y)
        tModel_features.append([dTree,features])
        
        
    return tModel_features# A LIST OF [SKLEARN TREE OBJECT, [COLS USED IN THAT TREE]] (SIZE OF OUTPUT LIST WILL BE (N_TREES*2))

5. Complete the following prediction function of random forest

In [6]:
# Make a prediction with a list of bagged trees
def random_forest_predict(test, trees):
    # WRITE YOUR CODE HERE
    # TEST IS THE TESTING DATASET THAT IS THE OUTPUT OF YOUR TRAIN TEST SPLIT FUNCTION
    # TREES ARE THE TRAINED TREES (OUTPUT OF THE RANDOM_FOREST_TRAIN FUNCTION)
    # RETURN TWO THINGS, 1 IS FINAL CLASS PREDICTIONS FOR EACH ROW IN TEST DATA, 2 IS THE PREDICTION BY EACH TREE FOR EACH ROW IN THE TEST DATA
    
    predictions = list()
    all_preds = list()
    
    for tree in trees:
        dTree = tree[0]
        features = tree[1].copy()
        features.remove('target')
        
        y_pred=dTree.predict(test[features])
        all_preds.append(y_pred)
        
    res = pd.DataFrame(data=all_preds)
    predictions=[]
    for col in res.columns:
        predictions.append(res[col].mode()[0])
    
    return predictions, all_preds

In [7]:
# importing the dataset
dataset = load_iris()
df = pd.DataFrame(dataset.data)
df["target"] = dataset.target
df

,0,1,2,3,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [8]:
# splitting the dataset into train and test
train, test = split_train_test(df, 0.3)

In [9]:
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)

In [10]:
trees = random_forest_train(train = train, n_trees = 10, max_depth = 3, sample_size = 0.8, n_features_ratio = 0.75)

In [11]:
trees

[[DecisionTreeClassifier(random_state=0), [2, 0, 'target']],
 [DecisionTreeClassifier(random_state=0), [0, 3, 'target']],
 [DecisionTreeClassifier(random_state=0), [0, 2, 'target']],
 [DecisionTreeClassifier(random_state=0), [2, 0, 'target']],
 [DecisionTreeClassifier(random_state=0), [3, 0, 'target']],
 [DecisionTreeClassifier(random_state=0), [0, 1, 'target']],
 [DecisionTreeClassifier(random_state=0), [3, 0, 'target']],
 [DecisionTreeClassifier(random_state=0), [0, 2, 'target']],
 [DecisionTreeClassifier(random_state=0), [2, 0, 'target']],
 [DecisionTreeClassifier(random_state=0), [3, 0, 'target']]]

In [12]:
predictions, all_preds = random_forest_predict(test.iloc[:,:-1], trees)

In [13]:
all_preds

[array([1, 0, 2, 2, 0, 1, 2, 1, 0, 0, 1, 2, 0, 0, 2, 2, 0, 0, 1, 1, 2, 1,
        0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 1, 0, 0, 1, 2, 2, 2, 2, 2, 0, 0, 0,
        0, 1, 0, 2, 0, 1, 0, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 1, 0, 1,
        2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 0, 1, 2, 0,
        0, 1, 1, 2, 2, 2, 0, 2, 0, 2, 1, 1, 1, 0, 2, 1, 1]),
 array([1, 0, 2, 2, 0, 1, 2, 1, 0, 0, 1, 2, 0, 0, 2, 2, 0, 0, 1, 1, 2, 1,
        0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 1, 0, 0, 1, 2, 2, 2, 2, 2, 0, 0, 0,
        0, 1, 0, 2, 0, 1, 0, 2, 0, 2, 2, 1, 2, 0, 2, 0, 2, 0, 0, 1, 0, 1,
        2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 0, 1, 2, 0,
        0, 1, 1, 1, 2, 2, 0, 2, 0, 2, 1, 1, 1, 0, 1, 1, 1]),
 array([1, 0, 2, 2, 0, 1, 2, 1, 0, 0, 1, 2, 0, 0, 2, 2, 0, 0, 1, 1, 2, 1,
        0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 1, 0, 0, 1, 2, 2, 2, 2, 2, 0, 0, 0,
        0, 1, 0, 2, 0, 1, 0, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 1, 0, 1,
        2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 0,

In [14]:
predictions

[1,
 0,
 2,
 2,
 0,
 1,
 2,
 1,
 0,
 0,
 1,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 1,
 1,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 1,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 1,
 0,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 2,
 0,
 0,
 1,
 1,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 1,
 1,
 1,
 0,
 2,
 1,
 1]

In [15]:
confusion_matrix(test.iloc[:,-1], predictions)

array([[36,  0,  0],
       [ 0, 35,  3],
       [ 0,  0, 31]])

In [16]:
accuracy_score(test.iloc[:,-1], predictions)

0.9714285714285714